# Команда "А"

- Артём Иванов
- Сергей Булаев
- Александр Хорин

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [3]:
from scipy import stats

d = stats.expon(0.1)

a = []

for i in range(56):
    a.append(d.rvs())

a.sort()

a = [1/x for x in a]
a = [x/a[-1] for x in a]
a = [int(x) for x in a]
summ = np.sum(a)
a = [x/summ for x in a]

a

[0.08455284552845528,
 0.05203252032520325,
 0.05040650406504065,
 0.03902439024390244,
 0.03577235772357724,
 0.03577235772357724,
 0.032520325203252036,
 0.032520325203252036,
 0.032520325203252036,
 0.032520325203252036,
 0.030894308943089432,
 0.02926829268292683,
 0.02926829268292683,
 0.027642276422764227,
 0.026016260162601626,
 0.026016260162601626,
 0.022764227642276424,
 0.022764227642276424,
 0.02113821138211382,
 0.02113821138211382,
 0.02113821138211382,
 0.01951219512195122,
 0.01951219512195122,
 0.01788617886178862,
 0.016260162601626018,
 0.016260162601626018,
 0.013008130081300813,
 0.013008130081300813,
 0.013008130081300813,
 0.011382113821138212,
 0.011382113821138212,
 0.00975609756097561,
 0.00975609756097561,
 0.00975609756097561,
 0.00975609756097561,
 0.008130081300813009,
 0.008130081300813009,
 0.0065040650406504065,
 0.0065040650406504065,
 0.0065040650406504065,
 0.0065040650406504065,
 0.004878048780487805,
 0.004878048780487805,
 0.004878048780487805,
 0

In [4]:
weights = np.array(a)

In [5]:
ss = pd.read_csv('./data/sample_submission_2.csv')

In [6]:
train = pd.read_csv('./data/train_2.csv')

In [7]:
keys = pd.read_csv('./data/key_2.csv')

In [21]:
ids = keys.Id.values
pages = keys.Page.values
d_pages = {}
dates = {}
for id, page in zip(ids, pages):
    date = page[-10:]
    dates[date] = True
    d_pages[id] = (page[:-11], page[-10:])

In [44]:
dates_indexes = {x: i for i, x in enumerate(dates.keys())}

In [40]:
len(dates.keys())

62

In [7]:
ids[:5]

array(['0b293039387a', '7114389dd824', '057b02ff1f09', 'bd2aca21caa3',
       'c0effb42cdd5'], dtype=object)

In [8]:
pages[:5]

array(['007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-13',
       '007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-14',
       '007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-15',
       '007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-16',
       '007_スペクター_ja.wikipedia.org_all-access_all-agents_2017-09-17'],
      dtype=object)

In [9]:
d_pages[ids[0]]

('007_スペクター_ja.wikipedia.org_all-access_all-agents', '2017-09-13')

# Медианы за всё время

In [8]:
medians = train.drop('Page', axis=1).apply(np.nanmedian, axis=1)
medians = np.round(medians)

In [84]:
np.max(medians)

19440903.0

# Медианы по дням недели

In [9]:
def week_medians(row):
    page = list(row)[5:]
    days = []
    for day in range(0, 7):
        days.append(np.round(np.nanmedian(page[day::7])))
    return days

/usr/local/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [ ]:
week_medians = train.apply(week_medians, axis=1)

In [31]:
week_medians.head()

0    [18.0, 17.0, 18.0, 19.0, 18.0, 19.0, 15.0]
1    [19.0, 19.0, 18.0, 19.0, 19.0, 21.0, 19.0]
2           [4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0]
3    [13.0, 14.0, 13.0, 15.0, 14.0, 14.0, 14.0]
4     [10.0, 10.0, 10.0, 10.0, 9.0, 12.0, 12.0]
dtype: object

In [10]:
weights.size

56

# Moving Average

In [15]:
def weighted_moving_average(row):
    num = 70
    wnum = 56
    pred = list(row.values)
    for i in range(0, num):
        pred_item = np.dot(pred[-wnum:], weights)
        pred.append(pred_item)
    return pred[-num:]

# pred = weighted_moving_average(page)
# np.dot(page[-weights.size:], weights)

In [73]:
moving_averages = train.fillna(medians).apply(weighted_moving_average, axis=1)

In [74]:
moving_averages.head(20)

0     [24.65528455284553, 24.103504527728205, 22.245...
1     [19.31219512195122, 19.668800317271465, 19.678...
2     [4.385365853658536, 4.446155066428713, 4.69078...
3     [14.674796747967479, 14.241747636988565, 14.46...
4     [17.40650406504065, 18.39578293343909, 18.9915...
5     [17.278048780487804, 17.46061471346421, 17.849...
6     [9.009756097560974, 9.539853261947254, 9.51471...
7     [50.213008130081306, 52.016606517284686, 52.70...
8     [37.96422764227643, 38.967421508361426, 39.809...
9     [27.398373983739837, 28.329103047128033, 25.98...
10    [24.730081300813012, 24.46134972569238, 24.426...
11    [11.611382113821138, 11.853026637583449, 11.18...
12    [5.978861788617886, 6.12354286469694, 6.058703...
13    [23.08130081300813, 23.504197237094324, 22.622...
14    [43.822764227642274, 40.8110939255734, 40.9376...
15    [34.489430894308946, 32.65282834291758, 30.575...
16    [29.565853658536582, 29.805798136030138, 30.66...
17    [11.520325203252032, 10.83174036618415, 10

In [75]:
page_predictions = {}
for page, m, wm, ma in zip(train.Page.values, medians, week_medians, moving_averages):
    xs1 = [m] * 62
    xs2 = (wm * 10)[2:65]
    xs3 = ma[:63]
    page_predictions[page] = list(map(np.average, zip(xs1, xs2, xs3)))

In [76]:
df = pd.DataFrame.from_dict(page_predictions).T

In [77]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
!vote_en.wikipedia.org_all-access_all-agents,6.400000,6.403902,6.159573,6.267258,6.191926,6.264585,6.514338,6.081997,6.036208,6.469516,...,6.045916,5.925722,5.776878,6.249854,6.010450,5.987848,6.281604,6.295643,5.974740,5.965615
!vote_en.wikipedia.org_all-access_spider,1.573984,1.582505,1.168276,1.183033,1.217340,1.288428,1.599071,1.248500,1.178644,1.149312,...,1.307414,1.300531,1.302904,1.617893,1.348094,1.335509,1.308720,1.318513,1.327417,1.335781
!vote_en.wikipedia.org_desktop_all-agents,4.107859,4.126463,3.654040,3.681295,3.833286,3.906165,3.834125,3.833247,3.849605,3.933126,...,3.723600,3.849292,3.724926,3.772109,3.850049,3.834877,3.804725,3.816557,3.830238,3.832581
"""Awaken,_My_Love!""_en.wikipedia.org_all-access_all-agents",1863.330623,1834.101351,1825.598534,1790.037986,1757.511925,1821.582023,1848.720696,1829.880047,1812.416189,1806.059834,...,1731.517459,1706.123468,1777.459489,1812.930022,1810.773625,1790.657598,1783.962025,1753.681145,1722.835611,1786.829035
"""Awaken,_My_Love!""_en.wikipedia.org_all-access_spider",23.983740,21.546857,22.152082,22.796748,21.745816,24.070853,24.538904,23.702858,21.290813,21.970964,...,22.804153,22.297835,24.023360,24.340642,23.743595,21.396038,22.375979,22.738594,22.062615,24.095005
"""Awaken,_My_Love!""_en.wikipedia.org_desktop_all-agents",1037.170732,1021.082662,1023.802580,992.286547,934.137554,982.662118,1021.480199,1019.739874,1009.861616,1013.390551,...,964.666987,909.947846,961.139272,1004.553683,1009.294501,997.471124,1002.295861,974.559703,917.804051,967.191721
"""European_Society_for_Clinical_Investigation""_en.wikipedia.org_all-access_all-agents",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""European_Society_for_Clinical_Investigation""_en.wikipedia.org_all-access_spider",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""European_Society_for_Clinical_Investigation""_en.wikipedia.org_desktop_all-agents",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Heroes""_(David_Bowie_album)_en.wikipedia.org_all-access_all-agents",636.798374,625.717829,633.826314,626.394267,625.487674,630.319079,638.460291,634.113948,625.597296,633.899928,...,631.034420,627.736765,631.558379,639.040306,635.561797,626.739446,635.755840,628.862879,628.011633,632.513096


In [81]:
ss_ids = ss.Id.values
ss_visits = ss.Visits.values

for i, ss_id in enumerate(ss_ids):
    page, date = d_pages[ss_id]
    try:
        ss_visits[i] = page_predictions[page][dates_indexes[date]]
    except:
        ss_visits[i] = 0

In [82]:
subm = pd.DataFrame({'Id': ss_ids, 'Visits': ss_visits})
subm.to_csv('submission_rnd.csv', index=False)